##### Needs to be refactored after data is available for more regions than just China

In [ ]:
import pandas as pd
import numpy as np

cred = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/scenario_credences/scenario.credences.csv', index_col = 0)
scenario = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_data/emit_data_weighted_by_affectability.csv', index_col = 0)

In [ ]:
clean_rcp = scenario['RCP'].str.replace('.', '')
cred['scenario'] = cred['scenario'].replace('Baseline', 'BL')

# make a column in the scenario data which will match the creedences data, merge on this column
scenario['cred_scen_match'] = scenario['SSP'] + '-' + clean_rcp
scen_and_prob = scenario.merge(cred, left_on = 'cred_scen_match', right_on = 'scenario', how = 'inner').fillna(0)

# make a copy of the merged data in order to weight it
scen_and_prob_weighted = scen_and_prob.copy()

# make active cols which should be weighted separate variable, matters b/c prob vars shouldn't be included
# in final output or get weighted by the following calculation
prob_cols_to_ignore = ['Country', 'SSP', 'RCP', 'cred_scen_match','scenario','credence','posterior','p']

# multiply each row of emissions (but not p itself) in the merged dataset of global emissions 
# by its scenario's p value
for col in scen_and_prob_weighted:
    if col not in prob_cols_to_ignore:
        scen_and_prob_weighted[col] = scen_and_prob_weighted[col] * scen_and_prob_weighted['p']

In [ ]:
# scen_and_prob_weighted # want the SUM of these columns to have expected value of emits for a region
# scen_and_prob # these are meaningful: they describe diff descrete potential worlds

In [ ]:
# remove the prob cols to avoid annoying indexing issues
scen_and_prob_weighted = scen_and_prob_weighted.drop(columns = ['cred_scen_match','scenario','credence','posterior','p'])

# calculate the last row of the whole DF, the global total of probability expected emissions for each year
# start at column 4, since the first three are country/rcp/ssp
global_totals = np.nansum(scen_and_prob_weighted.iloc[:,3:], axis = 0)

# add the appropriate "region", ssp, and rcp to the totals from each emission column
global_totals_row = ['Global','SSPtotal','RCPtotal'] + list(global_totals)

# male a dataframe from this list
global_expected = pd.DataFrame(global_totals_row).transpose()

# make the columns here the same as those from the emit df
global_expected.columns = scen_and_prob_weighted.columns

In [ ]:
# This code creates a row in the df where all the data from a given region is summed by column, the
# next cell appends that to the existing df. This cell can be uncommented where data is available for 
# more countries than China, and region mappings have been either explicitly included or added to the df.

# region_rows = []
# for region in scen_and_prob_weighted['region'].unique():
#     region_row_sum = np.nansum(scen_and_prob_weighted[scen_and_prob_weighted['region'] == region].iloc[:,3:819], axis = 0)
#     region_row_sum = [region] + ['SSPtotal'] + ['RCPtotal'] + list(region_row_sum)
#     region_rows.append(region_row_sum)

# region_expected = pd.DataFrame(region_rows, columns = scen_and_prob_weighted.columns[:-5])

In [ ]:
# appending the newly derived totals for regions and overall to the orginal (non-weighted) data
all_data = scen_and_prob.append(global_expected)
#uncomment with cell above: all_data = all_data.append(region_expected)

p = all_data.pop('p')
all_data.insert(3, 'p', p)

final = all_data.iloc[:,:-4].reset_index().drop(columns = 'index')
final.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/p_weighted_scen.csv')

In [ ]:
cred_split = cred['scenario'].str.split('-')
arr = cred_split.apply(pd.Series).values
SSPs = arr[:,0]
RCPs = np.char.replace(arr[:,1].astype(str), 'Baseline', 'BL')

rcp_dot = []
for rcp in RCPs:
    if 'BL' not in rcp:
        rcp = '.'.join(rcp)
        rcp_dot.append(rcp)
    elif 'BL' in rcp:
        rcp_dot.append(rcp)
        
output = pd.DataFrame(columns = ['credences.scenario', 'emit.data.scenario.SSP', 'emit.data.scenario.RCP'])
output['credences.scenario'] = cred['scenario']
output['emit.data.scenario.SSP'] = SSPs
output['emit.data.scenario.RCP'] = rcp_dot
output.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_scenarios_match.csv')